# 8-puzzle

A sliding puzzle that consists of a frame of 3*3 square tiles in random order with one tile missing(indicated by -1). <br>

The goal of the puzzle is to place the tiles in order (see below) by making sliding moves that use the empty tile. 

<b>Example:</b>

start state s: <br>
5 4 -1 <br>
6 1 8 <br>
7 3 2 <br>

goal state: <br>
1 2 3 <br>
8 -1 4 <br>
7 6 5 <br>

We will write Heuristic search algorithms to solve the 8-puzzle problem. 


In [ ]:
import numpy
import math

In [ ]:
print("hello")

# Operator and State Classes

## States:
A state of the board will be represented as a numpy 3x3 matrix. -1 is used to represent a empty tile.

## Operator
Transition from one state to another state happens as a result of applying operators - West, South, East, West. 

Following integers are used in the class below 
     0 - West, 1 - South, 2 - East and 3 - North'



In [2]:
#This class defines operators
class Operator:
    action = -1
    
    def __init__(self, act):
        self.action = act

#This class defines state 
class State:
    # This variable stores the tile configuration
    tiledArr = numpy.empty((3,3),dtype='int')
    
    # These two variables store the current location of empty tile
    freeTileI = 0
    freeTileJ = 0
    
    # This variable is to retrieve the path after finding a solution
    previous = None
    
    # This stores the cost from start node
    gVal = 0
    
    #Initializing the state with a given a tile configuration (arr)
    def __init__(self, arr):
        self.tiledArr = numpy.empty((3,3),dtype='int')
        for i in range(3):
            for j in range(3):
                self.tiledArr[i][j] = arr[i][j]
                
                #Empty tile is represented as a -1 in the 2D array
                if(self.tiledArr[i][j] == -1):
                    self.freeTileI = i
                    self.freeTileJ = j
    
    #This function returns a 1 if two tile configurations are the same and 0 otherwise
    def isEqual(self, s):
        for i in range(3):
            for j in range(3):
                if(self.tiledArr[i][j] != s.tiledArr[i][j]):
                    return 0
        return 1
    
    #This function applies an operator O on the current state (self) configuration 
    #and returns a new state configuration s1
    def applyOperator(self, o):
        s1 = State(self.tiledArr)
        s1.previous = self
        s1.gVal = self.gVal + 1
        #0 - West, 1 - South, 2 - East and 3 - North'
        if(o.action == 0):
            if(self.freeTileJ - 1 >= 0):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI][self.freeTileJ-1]
                s1.freeTileJ -= 1
        elif(o.action == 1):
            if(self.freeTileI + 1 < 3):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI+1][self.freeTileJ]
                s1.freeTileI += 1
        elif(o.action == 2):
            if(self.freeTileJ + 1 < 3):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI][self.freeTileJ+1]
                s1.freeTileJ += 1
        elif(o.action == 3):
            if(self.freeTileI - 1 >= 0):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI-1][self.freeTileJ]
                s1.freeTileI -= 1
        
        s1.tiledArr[s1.freeTileI][s1.freeTileJ] = -1
                
        return s1
    
    #This function prints the state configuration
    def printState(self):
        for i in range(3):
            for j in range(3):
                print("%d"%self.tiledArr[i][j],end=' ')
            print()
        print("*******")

#  Utility Functions

Next we define some utility functions which can be used by Heuristic Search algorithms. These functions helps in checking whether a state is present in a given list, to retrieve path from a given state.


In [3]:
#Function that indicates whether a state is present in a list
def isPresentStateInList(state, searchList):
    for elem in searchList:
        if(state.isEqual(elem) == 1):
            return 1

    return 0

#Function that indicates whether a state is present in a priority list
def isPresentStateInPriorityList(state, searchList):
    for [elem, dist] in searchList:
        if(state.isEqual(elem) == 1):
            return 1
        
    return 0

def insertStateInPriorityQueue(searchList, state, distanceToGoal):
    index = -1
    for i in range(len(searchList)):
        if(distanceToGoal < searchList[i][1]):
            index = i
            break
    
    if(len(searchList) == 0 or index == -1):
        searchList.append([state,distanceToGoal])
    else:
        searchList.insert(index, [state,distanceToGoal])

#Reinserts the element in priority queue if the new value is less than the 
#value present in queue.
def checkAndUpdateStateInPriorityQueue(searchList,state,distanceToGoal):
    index = -1
    for i in range(len(searchList)):
        if(state.isEqual(searchList[i][0])):
            if distanceToGoal < searchList[i][1]:
                    index = i
            break
            
    if index!=-1:
        searchList.remove([searchList[index][0],searchList[index][1]])
        insertStateInPriorityQueue(searchList, state, distanceToGoal)
        

        
        
def retrievePathFromState(state):
    visitedstatelist = []
    visitedstatelist.append(state)
    state.printState()
    prev = state.previous
    counter = 0
    while(prev != None):
        visitedstatelist.append(prev)
        #prev.printState()
        prev = prev.previous
        counter +=1
    visitedstatelist.reverse()
    for i in range(len(visitedstatelist)):
        visitedstatelist[i].printState()
    print("Size of path is ",counter,state.gVal)

# Heuristic functions:
We define 2 heuristic functions for the 8 puzzle problem. These heuristic functions can be used by informed search algorithms such as A* and Best First. We use H1 and H2 to denote the two heuristic functions. 

## H1 - Number of Tiles in Wrong Position
This heuristic function compares the given state and goal state to identify the number of tiles in the wrong position.

## H2 - Manhattan distance
This heuristic function computes the sum of manhattan distance of each tile in given state to its position in goal state. 

Both these heuristics are admissible and consistent. 

In [4]:
#Number of tiles in wrong position
def calculateH1Value(current, goal):
    counter = 0
    for i in range(3):
        for j in range(3):
            if(current.tiledArr[i][j] != goal.tiledArr[i][j]):
                counter+=1

    return counter
# Sum of manhattan distance of each tile from its goal position
def calculateH2Value(current, goal):
    xposcurrent = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    yposcurrent = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    xposgoal =  [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    yposgoal =  [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    for i in range(3):
        for j in range(3):
            if current.tiledArr[i][j]!=-1:
                xposcurrent[current.tiledArr[i][j]] = i
                yposcurrent[current.tiledArr[i][j]] = j
            if goal.tiledArr[i][j]!=-1:
                xposgoal[goal.tiledArr[i][j]] = i
                yposgoal[goal.tiledArr[i][j]] = j
    totdist=0
    for i in range(1,9):
        totdist+= (math.fabs(xposcurrent[i]-xposgoal[i])+math.fabs(yposcurrent[i]-yposgoal[i]))
    
    return totdist

# Heuristic Search Algorithms

We now implement four heuristic search algorithms. 

Uninformed Search Algorithms 
1. Depth First Search (DFS)
2. Breadth First Search (BFS)

Informed Search Algorithms
1. Best First Search 
2. A* Search 

All the search algorithms  use open list and the closed list. The open list keeps track of
those nodes that need to be examined, while the closed list keeps track of nodes that have already been
examined. 

For informed searches, open list is a priority queue. 

Initially, the open list contains just the initial node, and the closed list is empty. 

# Depth First Search

Algorithm: 

1. Get as input the start and the final state.
2. Put the starting state into open list.
3. Retrieve the first state, eliminate it from open list and add it to closed list.
4. Generate all possible states we can get from this state by applying operators. 
5. If any of these is the goal state, we are done, retrieve the path from start state to this state. 
6. Else if this state is not present in open and closed list add it to the beginning of the open list. 
7. Jump to step 3. 



In [6]:
def searchDFS(start, goal):
    openList = []
    closedList = []
    
    openList.append(start)
    
    while(len(openList) != 0):
        first = openList[0]
        openList.remove(first)
        closedList.append(first)
        
        for i in range(4):
            o = Operator(i)
            succState = first.applyOperator(o)
            
            if(succState.isEqual(goal) == 1):
                retrievePathFromState(succState)
                return
            
            if(isPresentStateInList(succState, openList) == 0 and isPresentStateInList(succState,closedList) == 0):
                openList.insert(0, succState)
            

start = State([[-1,5,6],[4,3,8],[7,1,2]])
goal = State([[4,5,6],[7,3,8],[-1,1,2]])                    
searchDFS(start, goal)



4 5 6 
7 3 8 
-1 1 2 
*******
-1 5 6 
4 3 8 
7 1 2 
*******
5 -1 6 
4 3 8 
7 1 2 
*******
5 6 -1 
4 3 8 
7 1 2 
*******
5 6 8 
4 3 -1 
7 1 2 
*******
5 6 8 
4 3 2 
7 1 -1 
*******
5 6 8 
4 3 2 
7 -1 1 
*******
5 6 8 
4 -1 2 
7 3 1 
*******
5 -1 8 
4 6 2 
7 3 1 
*******
5 8 -1 
4 6 2 
7 3 1 
*******
5 8 2 
4 6 -1 
7 3 1 
*******
5 8 2 
4 6 1 
7 3 -1 
*******
5 8 2 
4 6 1 
7 -1 3 
*******
5 8 2 
4 -1 1 
7 6 3 
*******
5 -1 2 
4 8 1 
7 6 3 
*******
5 2 -1 
4 8 1 
7 6 3 
*******
5 2 1 
4 8 -1 
7 6 3 
*******
5 2 1 
4 8 3 
7 6 -1 
*******
5 2 1 
4 8 3 
7 -1 6 
*******
5 2 1 
4 -1 3 
7 8 6 
*******
5 -1 1 
4 2 3 
7 8 6 
*******
5 1 -1 
4 2 3 
7 8 6 
*******
5 1 3 
4 2 -1 
7 8 6 
*******
5 1 3 
4 2 6 
7 8 -1 
*******
5 1 3 
4 2 6 
7 -1 8 
*******
5 1 3 
4 -1 6 
7 2 8 
*******
5 -1 3 
4 1 6 
7 2 8 
*******
5 3 -1 
4 1 6 
7 2 8 
*******
5 3 6 
4 1 -1 
7 2 8 
*******
5 3 6 
4 1 8 
7 2 -1 
*******
5 3 6 
4 1 8 
7 -1 2 
*******
5 3 6 
4 1 8 
-1 7 2 
*******
5 3 6 
-1 1 8 
4 7 2 
*******
-1 3 6 
5 

# Breadth First Search

Algorithm: 
1. Get as input the start and the final state.
2. Put the starting state into open list.
3. Retrieve the first state, eliminate it from open list and add it to closed list.
4. Generate all possible states we can get from this state by applying operators. 
5. If any of these is the goal state, we are done, retrieve the path from start state to this state. 
6. Else if this state is not present in open and closed list add it to the end of the open list. 
7. Jump to step 3. 


In [ ]:
def searchBFS(start, goal):
    pass

# Best First Search 

Algorithm:

1. Get as input the start and the final state.
2. Put the starting state into open list with its priority value as the heuristic value to reach the goal.
3. Retrieve the first state, eliminate it from open list and add it to closed list.
4. If this is the goal state, we are done. retrieve the path from start state to this state.
5. If not then  Generate all possible states we can get from this state by applying operators. 
6. If these states are not present in open and closed list add it to the the open list with the priority as the heuristic value of reaching goal from this state. 
7. If the state is present in open list, check if the value(priority) needs to be updated. If yes, update the value and reinsert into queue at correcct position
8. Jump to step 3. 

In [ ]:
def searchBestFirst(start, goal):
    pass
            

# A* Search

A* search uses the gval - the cost of getting to the state from the start state. gval of start state is 0. 

In this code the State class internally increments the gval of the state on applying operator. 

Algorithm:


1. Get as input the start and the final state.
2. Put the starting state into open list with its priority value as the sum of gval of current state and heuristic value to reach the goal.
3. Retrieve the first state, eliminate it from open list and add it to closed list.
4. If this is the goal state, we are done. retrieve the path from start state to this state.
5. If not then  Generate all possible states we can get from this state by applying operators. 
6. If these states are not present in open and closed list add it to the the open list with the priority as the sum of gval of state and heuristic value of reaching goal from this state. 
7. If the state is present in open list, check if the value(priority) needs to be updated. If yes, update the value and reinsert into queue at correcct position
8. Jump to step 3. 

In [ ]:
def searchAStar(start, goal):
    pass

# Try Search Functions

We try all the Heuristic Search methods for different scenarios. We can see that informed searches, search for lesser or equal nodes to uninformed searches.



In [ ]:


#DFS explores less nodes than BFS, obtains same solution
start = State([[4,5,6],[7,3,8],[1,2,-1]])
goal = State([[4,-1,5],[7,3,6],[1,2,8]])

print("Running A*")

searchAStar(start, goal)
print("Finished A*")

print("Running Best First")
searchBestFirst(start, goal)
print("Finished Best First")

print("Running Breadth First")
searchBFS(start, goal)
print("Finished Breadth First")

print("Running Depth First")
searchDFS(start, goal)
print("Finished Depth First")


In [ ]:
#Best first searches for less nodes as compared to A*. Solution obtained is same
#BFS takes more than 3000 nodes. 
#DFS could not find a solution after searching for 5000 nodes
start = State([[4,5,6],[-1,3,8],[7,1,2]])
goal = State([[6,8,2],[5,3,1],[4,-1,7]])
print("Running A*")

searchAStar(start, goal)
print("Finished A*")

print("Running Best First")
searchBestFirst(start, goal)
print("Finished Best First")

print("Running Breadth First")
searchBFS(start, goal)
print("Finished Breadth First")

print("Running Depth First")
searchDFS(start, goal)
print("Finished Depth First")

In [ ]:

# Best first and A* same number of nodes explored. Solution obtained is same
start = State([[2,8,3],[1,6,4],[7,-1,5]])
goal = State([[1,2,3],[8,-1,4],[7,6,5]])

print("Running A*")

searchAStar(start, goal)
print("Finished A*")

print("Running Best First")
searchBestFirst(start, goal)
print("Finished Best First")

print("Running Breadth First")
searchBFS(start, goal)
print("Finished Breadth First")

print("Running Depth First")
searchDFS(start, goal)
print("Finished Depth First")

In [ ]:

#Best first explores less nodes but provides non optimal solution.  A* explores around 6000 nodes before finding a solution.
start = State([[5,-1,8],[4,2,1],[7,3,6]])
goal = State([[1,2,3],[4,5,6],[7,8,-1]])

print("Running A*")

searchAStar(start, goal)
print("Finished A*")

print("Running Best First")
searchBestFirst(start, goal)
print("Finished Best First")

print("Running Breadth First")
searchBFS(start, goal)
print("Finished Breadth First")

print("Running Depth First")
searchDFS(start, goal)
print("Finished Depth First")

In [ ]:

#DFS can not find a path after searching 5000 nodes. BFS finds a path after searching 2 nodes
start = State([[4,5,6],[3,8,-1],[7,1,2]])
goal = State([[4,5,6],[-1,3,8],[7,1,2]])


print("Running A*")

searchAStar(start, goal)
print("Finished A*")

print("Running Best First")
searchBestFirst(start, goal)
print("Finished Best First")

print("Running Breadth First")
searchBFS(start, goal)
print("Finished Breadth First")

print("Running Depth First")
searchDFS(start, goal)
print("Finished Depth First")

In [ ]:

#DFS - explores lot more nodes and provides a non optimal path
start = State([[-1,5,6],[4,3,8],[7,1,2]])
goal = State([[4,5,6],[7,3,8],[-1,1,2]])


print("Running A*")

searchAStar(start, goal)
print("Finished A*")

print("Running Best First")
searchBestFirst(start, goal)
print("Finished Best First")

print("Running Breadth First")
searchBFS(start, goal)
print("Finished Breadth First")

print("Running Depth First")
searchDFS(start, goal)
print("Finished Depth First")